In [1]:
import pandas as pd
import numpy as np
##import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.metrics import roc_auc_score, accuracy_score,f1_score

In [2]:
insure= pd.read_csv(r"E:\assignment\insurance\train.csv")

In [3]:
insure.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,1,Male,44,1,28.0,0,> 2 Years,Yes,40454.0,26.0,217,1
1,2,Male,76,1,3.0,0,1-2 Year,No,33536.0,26.0,183,0
2,3,Male,47,1,28.0,0,> 2 Years,Yes,38294.0,26.0,27,1
3,4,Male,21,1,11.0,1,< 1 Year,No,28619.0,152.0,203,0
4,5,Female,29,1,41.0,1,< 1 Year,No,27496.0,152.0,39,0


In [4]:
insure.shape

(381109, 12)

In [5]:
bhool_col= ["Driving_License","Previously_Insured","Region_Code" ]
int_col=["Age", "Region_Code","Annual_Premium","Vintage"]
target= "Response"
feat_colmns= (bhool_col+int_col+[target])
feat_colmns

['Driving_License',
 'Previously_Insured',
 'Region_Code',
 'Age',
 'Region_Code',
 'Annual_Premium',
 'Vintage',
 'Response']

In [6]:
from sklearn.model_selection import train_test_split
n_total = len(insure)

# Train, val and test split follows
# Rory Mitchell, Andrey Adinets, Thejaswi Rao, and Eibe Frank.
# Xgboost: Scalable GPU accelerated learning. arXiv:1806.11248, 2018.

train_val_indices, test_indices = train_test_split(
    range(n_total), test_size=0.2, random_state=0)
train_indices, valid_indices = train_test_split(
    train_val_indices, test_size=0.2 / 0.6, random_state=0)


In [7]:
from sklearn.preprocessing import LabelEncoder
cat_columns=[]
cat_dim={}
for col in insure.columns[insure.dtypes=="object"]:
    print(col, insure[col].nunique())
    l_enc = LabelEncoder()
    insure[col] = insure[col].fillna("VV_likely")
    insure[col] = l_enc.fit_transform(insure[col].values)
    cat_columns.append(col)
    cat_dim[col] = len(l_enc.classes_)
for col in insure.columns[insure.dtypes == 'float64']:
    insure.fillna(insure.loc[train_indices, col].mean(), inplace=True)    
   

Gender 2
Vehicle_Age 3
Vehicle_Damage 2


In [8]:
len(l_enc.classes_)

2

In [12]:
unused_feat = []

features = [ col for col in insure.columns if col not in unused_feat+[target]] 

cat_idxs = [ i for i, f in enumerate(features) if f in cat_columns]

cat_dims = [ cat_dim[f] for i, f in enumerate(features) if f in cat_columns]


In [21]:
import torch
from pytorch_tabnet.tab_model import TabNetClassifier
cmd = TabNetClassifier(cat_idxs=cat_idxs,
                       cat_dims=cat_dims,
                       cat_emb_dim=1,
                       optimizer_fn=torch.optim.Adam,
                       optimizer_params=dict(lr=2e-2),
                       scheduler_params={"step_size":50, # how to use learning rate scheduler
                                         "gamma":0.9},
                       scheduler_fn=torch.optim.lr_scheduler.StepLR,
                       mask_type='entmax' # "sparsemax"
)

Device used : cuda


In [25]:
X_train = insure[features].values[train_indices]
y_train = insure[target].values[train_indices]

In [26]:
X_valid = insure[features].values[valid_indices]
y_valid = insure[target].values[valid_indices]

In [27]:
X_test = insure[features].values[test_indices]
y_test = insure[target].values[test_indices]

In [26]:
cmd.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    eval_name=['train', 'valid'],
    max_epochs= 25, patience=100,
    batch_size=1024, virtual_batch_size=128, weights=1,drop_last=False
)

epoch 0  | loss: 0.43178 | train_auc: 0.85075 | valid_auc: 0.84959 |  0:00:35s
epoch 1  | loss: 0.42838 | train_auc: 0.85103 | valid_auc: 0.84965 |  0:01:11s
epoch 2  | loss: 0.429   | train_auc: 0.85013 | valid_auc: 0.84907 |  0:01:47s
epoch 3  | loss: 0.43116 | train_auc: 0.84978 | valid_auc: 0.84831 |  0:02:22s
epoch 4  | loss: 0.43089 | train_auc: 0.85108 | valid_auc: 0.84959 |  0:02:57s
epoch 5  | loss: 0.42892 | train_auc: 0.85139 | valid_auc: 0.8498  |  0:03:33s
epoch 6  | loss: 0.42818 | train_auc: 0.8518  | valid_auc: 0.85014 |  0:04:08s
epoch 7  | loss: 0.42724 | train_auc: 0.85134 | valid_auc: 0.85036 |  0:04:43s
epoch 8  | loss: 0.42695 | train_auc: 0.8514  | valid_auc: 0.84974 |  0:05:18s
epoch 9  | loss: 0.42643 | train_auc: 0.85248 | valid_auc: 0.85013 |  0:05:53s
epoch 10 | loss: 0.42858 | train_auc: 0.8518  | valid_auc: 0.8493  |  0:06:29s
epoch 11 | loss: 0.42804 | train_auc: 0.85286 | valid_auc: 0.85039 |  0:07:05s
epoch 12 | loss: 0.42738 | train_auc: 0.85295 | vali

In [46]:
y_pred = cmd.predict_proba(X_test)[:,1]

In [56]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score


In [50]:
y_pred1= y_pred.astype(np.int)

<ipython-input-50-97f791dbbc92>:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_pred1= y_pred.astype(np.int)


In [51]:
y_test, y_pred1

(array([0, 0, 0, ..., 0, 0, 0], dtype=int64), array([0, 0, 0, ..., 0, 0, 0]))

In [53]:
roc_auc_score(y_test, y_pred1)

0.5

In [57]:
accuracy_score(y_test, y_pred1)

0.8769908950171866

In [8]:
insure.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,1,1,44,1,28.0,0,2,1,40454.0,26.0,217,1
1,2,1,76,1,3.0,0,0,0,33536.0,26.0,183,0
2,3,1,47,1,28.0,0,2,1,38294.0,26.0,27,1
3,4,1,21,1,11.0,1,1,0,28619.0,152.0,203,0
4,5,0,29,1,41.0,1,1,0,27496.0,152.0,39,0


In [9]:
x= insure.drop("Response", axis=1)
y= insure["Response"]
train_x, test_x, train_y, test_y= train_test_split(x, y, test_size=0.25)
train_x.shape, train_y.shape, test_x.shape, test_y.shape

((285831, 11), (285831,), (95278, 11), (95278,))

In [10]:
from catboost import CatBoostClassifier
model= CatBoostClassifier( iterations=2500)
model.fit(train_x,train_y)


Learning rate set to 0.049749
0:	learn: 0.6026350	total: 210ms	remaining: 8m 44s
1:	learn: 0.5316481	total: 249ms	remaining: 5m 10s
2:	learn: 0.4774389	total: 286ms	remaining: 3m 58s
3:	learn: 0.4355599	total: 325ms	remaining: 3m 22s
4:	learn: 0.4221928	total: 361ms	remaining: 2m 59s
5:	learn: 0.3952582	total: 398ms	remaining: 2m 45s
6:	learn: 0.3719827	total: 449ms	remaining: 2m 39s
7:	learn: 0.3539013	total: 488ms	remaining: 2m 32s
8:	learn: 0.3398178	total: 526ms	remaining: 2m 25s
9:	learn: 0.3297919	total: 563ms	remaining: 2m 20s
10:	learn: 0.3206373	total: 602ms	remaining: 2m 16s
11:	learn: 0.3132123	total: 640ms	remaining: 2m 12s
12:	learn: 0.3075205	total: 685ms	remaining: 2m 11s
13:	learn: 0.3023114	total: 725ms	remaining: 2m 8s
14:	learn: 0.2980310	total: 763ms	remaining: 2m 6s
15:	learn: 0.2943651	total: 803ms	remaining: 2m 4s
16:	learn: 0.2913461	total: 845ms	remaining: 2m 3s
17:	learn: 0.2887345	total: 883ms	remaining: 2m 1s
18:	learn: 0.2866665	total: 928ms	remaining: 2m 1

157:	learn: 0.2649507	total: 7.92s	remaining: 1m 57s
158:	learn: 0.2649384	total: 7.98s	remaining: 1m 57s
159:	learn: 0.2649207	total: 8.04s	remaining: 1m 57s
160:	learn: 0.2649007	total: 8.08s	remaining: 1m 57s
161:	learn: 0.2648895	total: 8.12s	remaining: 1m 57s
162:	learn: 0.2648737	total: 8.16s	remaining: 1m 57s
163:	learn: 0.2648587	total: 8.2s	remaining: 1m 56s
164:	learn: 0.2648379	total: 8.24s	remaining: 1m 56s
165:	learn: 0.2648235	total: 8.28s	remaining: 1m 56s
166:	learn: 0.2648149	total: 8.31s	remaining: 1m 56s
167:	learn: 0.2648007	total: 8.35s	remaining: 1m 55s
168:	learn: 0.2647863	total: 8.4s	remaining: 1m 55s
169:	learn: 0.2647628	total: 8.44s	remaining: 1m 55s
170:	learn: 0.2647455	total: 8.48s	remaining: 1m 55s
171:	learn: 0.2647337	total: 8.52s	remaining: 1m 55s
172:	learn: 0.2647219	total: 8.56s	remaining: 1m 55s
173:	learn: 0.2646954	total: 8.6s	remaining: 1m 54s
174:	learn: 0.2646787	total: 8.64s	remaining: 1m 54s
175:	learn: 0.2646590	total: 8.68s	remaining: 1m 

312:	learn: 0.2628992	total: 14.6s	remaining: 1m 41s
313:	learn: 0.2628848	total: 14.7s	remaining: 1m 42s
314:	learn: 0.2628694	total: 14.7s	remaining: 1m 42s
315:	learn: 0.2628567	total: 14.8s	remaining: 1m 42s
316:	learn: 0.2628476	total: 14.9s	remaining: 1m 42s
317:	learn: 0.2628377	total: 14.9s	remaining: 1m 42s
318:	learn: 0.2628266	total: 15s	remaining: 1m 42s
319:	learn: 0.2628138	total: 15.1s	remaining: 1m 42s
320:	learn: 0.2627998	total: 15.1s	remaining: 1m 42s
321:	learn: 0.2627864	total: 15.2s	remaining: 1m 42s
322:	learn: 0.2627766	total: 15.3s	remaining: 1m 42s
323:	learn: 0.2627679	total: 15.3s	remaining: 1m 42s
324:	learn: 0.2627546	total: 15.4s	remaining: 1m 43s
325:	learn: 0.2627385	total: 15.5s	remaining: 1m 43s
326:	learn: 0.2627257	total: 15.5s	remaining: 1m 43s
327:	learn: 0.2627164	total: 15.6s	remaining: 1m 43s
328:	learn: 0.2627030	total: 15.6s	remaining: 1m 42s
329:	learn: 0.2626901	total: 15.6s	remaining: 1m 42s
330:	learn: 0.2626794	total: 15.7s	remaining: 1m

470:	learn: 0.2612387	total: 23s	remaining: 1m 39s
471:	learn: 0.2612255	total: 23s	remaining: 1m 39s
472:	learn: 0.2612170	total: 23.1s	remaining: 1m 38s
473:	learn: 0.2612032	total: 23.1s	remaining: 1m 38s
474:	learn: 0.2611944	total: 23.2s	remaining: 1m 38s
475:	learn: 0.2611843	total: 23.3s	remaining: 1m 38s
476:	learn: 0.2611782	total: 23.3s	remaining: 1m 38s
477:	learn: 0.2611691	total: 23.4s	remaining: 1m 38s
478:	learn: 0.2611584	total: 23.4s	remaining: 1m 38s
479:	learn: 0.2611468	total: 23.5s	remaining: 1m 38s
480:	learn: 0.2611404	total: 23.5s	remaining: 1m 38s
481:	learn: 0.2611313	total: 23.6s	remaining: 1m 38s
482:	learn: 0.2611213	total: 23.6s	remaining: 1m 38s
483:	learn: 0.2611144	total: 23.7s	remaining: 1m 38s
484:	learn: 0.2611067	total: 23.7s	remaining: 1m 38s
485:	learn: 0.2610984	total: 23.8s	remaining: 1m 38s
486:	learn: 0.2610891	total: 23.8s	remaining: 1m 38s
487:	learn: 0.2610821	total: 23.9s	remaining: 1m 38s
488:	learn: 0.2610741	total: 23.9s	remaining: 1m 3

627:	learn: 0.2597814	total: 31.2s	remaining: 1m 33s
628:	learn: 0.2597711	total: 31.3s	remaining: 1m 33s
629:	learn: 0.2597628	total: 31.4s	remaining: 1m 33s
630:	learn: 0.2597573	total: 31.4s	remaining: 1m 33s
631:	learn: 0.2597475	total: 31.5s	remaining: 1m 32s
632:	learn: 0.2597374	total: 31.5s	remaining: 1m 32s
633:	learn: 0.2597301	total: 31.6s	remaining: 1m 32s
634:	learn: 0.2597208	total: 31.6s	remaining: 1m 32s
635:	learn: 0.2597087	total: 31.7s	remaining: 1m 32s
636:	learn: 0.2596995	total: 31.7s	remaining: 1m 32s
637:	learn: 0.2596905	total: 31.8s	remaining: 1m 32s
638:	learn: 0.2596815	total: 31.8s	remaining: 1m 32s
639:	learn: 0.2596723	total: 31.9s	remaining: 1m 32s
640:	learn: 0.2596660	total: 31.9s	remaining: 1m 32s
641:	learn: 0.2596581	total: 32s	remaining: 1m 32s
642:	learn: 0.2596481	total: 32s	remaining: 1m 32s
643:	learn: 0.2596376	total: 32.1s	remaining: 1m 32s
644:	learn: 0.2596270	total: 32.1s	remaining: 1m 32s
645:	learn: 0.2596181	total: 32.2s	remaining: 1m 3

...

1869:	learn: 0.2506986	total: 1m 42s	remaining: 34.5s
1870:	learn: 0.2506938	total: 1m 42s	remaining: 34.5s
1871:	learn: 0.2506859	total: 1m 42s	remaining: 34.4s
1872:	learn: 0.2506815	total: 1m 42s	remaining: 34.4s
1873:	learn: 0.2506750	total: 1m 42s	remaining: 34.3s
1874:	learn: 0.2506699	total: 1m 42s	remaining: 34.3s
1875:	learn: 0.2506604	total: 1m 42s	remaining: 34.2s
1876:	learn: 0.2506481	total: 1m 42s	remaining: 34.1s
1877:	learn: 0.2506365	total: 1m 42s	remaining: 34.1s
1878:	learn: 0.2506292	total: 1m 42s	remaining: 34s
1879:	learn: 0.2506246	total: 1m 43s	remaining: 34s
1880:	learn: 0.2506176	total: 1m 43s	remaining: 33.9s
1881:	learn: 0.2506109	total: 1m 43s	remaining: 33.9s
1882:	learn: 0.2506055	total: 1m 43s	remaining: 33.8s
1883:	learn: 0.2505991	total: 1m 43s	remaining: 33.8s
1884:	learn: 0.2505943	total: 1m 43s	remaining: 33.7s
1885:	learn: 0.2505916	total: 1m 43s	remaining: 33.7s
1886:	learn: 0.2505822	total: 1m 43s	remaining: 33.6s
1887:	learn: 0.2505725	total: 1m

2023:	learn: 0.2496908	total: 1m 51s	remaining: 26.2s
2024:	learn: 0.2496847	total: 1m 51s	remaining: 26.1s
2025:	learn: 0.2496785	total: 1m 51s	remaining: 26s
2026:	learn: 0.2496721	total: 1m 51s	remaining: 26s
2027:	learn: 0.2496650	total: 1m 51s	remaining: 25.9s
2028:	learn: 0.2496596	total: 1m 51s	remaining: 25.9s
2029:	learn: 0.2496536	total: 1m 51s	remaining: 25.8s
2030:	learn: 0.2496476	total: 1m 51s	remaining: 25.8s
2031:	learn: 0.2496433	total: 1m 51s	remaining: 25.7s
2032:	learn: 0.2496370	total: 1m 51s	remaining: 25.7s
2033:	learn: 0.2496280	total: 1m 51s	remaining: 25.6s
2034:	learn: 0.2496217	total: 1m 51s	remaining: 25.5s
2035:	learn: 0.2496143	total: 1m 51s	remaining: 25.5s
2036:	learn: 0.2496087	total: 1m 51s	remaining: 25.4s
2037:	learn: 0.2496060	total: 1m 51s	remaining: 25.4s
2038:	learn: 0.2496042	total: 1m 51s	remaining: 25.3s
2039:	learn: 0.2495989	total: 1m 52s	remaining: 25.3s
2040:	learn: 0.2495934	total: 1m 52s	remaining: 25.2s
2041:	learn: 0.2495851	total: 1m

2176:	learn: 0.2487514	total: 1m 59s	remaining: 17.7s
2177:	learn: 0.2487457	total: 1m 59s	remaining: 17.7s
2178:	learn: 0.2487410	total: 1m 59s	remaining: 17.6s
2179:	learn: 0.2487363	total: 1m 59s	remaining: 17.6s
2180:	learn: 0.2487317	total: 1m 59s	remaining: 17.5s
2181:	learn: 0.2487243	total: 1m 59s	remaining: 17.4s
2182:	learn: 0.2487179	total: 1m 59s	remaining: 17.4s
2183:	learn: 0.2487130	total: 1m 59s	remaining: 17.3s
2184:	learn: 0.2487041	total: 1m 59s	remaining: 17.3s
2185:	learn: 0.2486981	total: 1m 59s	remaining: 17.2s
2186:	learn: 0.2486905	total: 1m 59s	remaining: 17.2s
2187:	learn: 0.2486838	total: 1m 59s	remaining: 17.1s
2188:	learn: 0.2486781	total: 2m	remaining: 17.1s
2189:	learn: 0.2486727	total: 2m	remaining: 17s
2190:	learn: 0.2486652	total: 2m	remaining: 16.9s
2191:	learn: 0.2486617	total: 2m	remaining: 16.9s
2192:	learn: 0.2486514	total: 2m	remaining: 16.8s
2193:	learn: 0.2486452	total: 2m	remaining: 16.8s
2194:	learn: 0.2486405	total: 2m	remaining: 16.7s
2195

2335:	learn: 0.2477957	total: 2m 8s	remaining: 8.99s
2336:	learn: 0.2477893	total: 2m 8s	remaining: 8.94s
2337:	learn: 0.2477804	total: 2m 8s	remaining: 8.88s
2338:	learn: 0.2477722	total: 2m 8s	remaining: 8.83s
2339:	learn: 0.2477662	total: 2m 8s	remaining: 8.78s
2340:	learn: 0.2477601	total: 2m 8s	remaining: 8.72s
2341:	learn: 0.2477548	total: 2m 8s	remaining: 8.67s
2342:	learn: 0.2477487	total: 2m 8s	remaining: 8.62s
2343:	learn: 0.2477429	total: 2m 8s	remaining: 8.56s
2344:	learn: 0.2477392	total: 2m 8s	remaining: 8.51s
2345:	learn: 0.2477354	total: 2m 8s	remaining: 8.46s
2346:	learn: 0.2477298	total: 2m 8s	remaining: 8.4s
2347:	learn: 0.2477234	total: 2m 8s	remaining: 8.35s
2348:	learn: 0.2477174	total: 2m 9s	remaining: 8.29s
2349:	learn: 0.2477098	total: 2m 9s	remaining: 8.24s
2350:	learn: 0.2477038	total: 2m 9s	remaining: 8.18s
2351:	learn: 0.2477000	total: 2m 9s	remaining: 8.13s
2352:	learn: 0.2476943	total: 2m 9s	remaining: 8.07s
2353:	learn: 0.2476887	total: 2m 9s	remaining: 

2489:	learn: 0.2468936	total: 2m 16s	remaining: 550ms
2490:	learn: 0.2468917	total: 2m 16s	remaining: 495ms
2491:	learn: 0.2468862	total: 2m 17s	remaining: 440ms
2492:	learn: 0.2468800	total: 2m 17s	remaining: 385ms
2493:	learn: 0.2468732	total: 2m 17s	remaining: 330ms
2494:	learn: 0.2468672	total: 2m 17s	remaining: 275ms
2495:	learn: 0.2468615	total: 2m 17s	remaining: 220ms
2496:	learn: 0.2468539	total: 2m 17s	remaining: 165ms
2497:	learn: 0.2468487	total: 2m 17s	remaining: 110ms
2498:	learn: 0.2468450	total: 2m 17s	remaining: 55ms
2499:	learn: 0.2468410	total: 2m 17s	remaining: 0us


In [11]:
pred= model.predict(test_x)
roc_auc_score(test_y, pred), accuracy_score(test_y, pred)

(0.5134302765272047, 0.87682361090703)